# Execute this code first

In [6]:
import random as rnd
import operator
import os
import matplotlib.pyplot as plt                 # import matplotlib
import numpy as np                              # import numpy
import time
import copy
import json

# This works only if notebook is in same folder
import dm_bargain
import dm_travel
import dm_agents
import dm_env as env
import dm_utils as dm
import dm_sim_period as simp
import dm_process_results as pr
import env_make_agents as mkt

def make_sim(sim_name, num_periods, num_weeks,
             num_rounds, grid_size,
             num_traders, num_units,
             lower_bound, upper_bound,
             trader_objects):
    """Runs one complete simulation and returns data in
        effs[treatment][trial]
    """ 

    # data table for simulation
    data = {}

    # make agents
    agent_maker = mkt.MakeAgents(num_traders, trader_objects, num_units, 
                                grid_size, lower_bound, upper_bound)
    agent_maker.make_agents()
    agent_maker.set_locations(grid_size)
    agents = agent_maker.get_agents()
  
    # set up market
    agent_maker.make_market(sim_name)
    market = agent_maker.get_market()

    # run sim
    for week in range(num_weeks):
        for agent in agents:
            agent.start(None)
        contracts = []
        sim_grids = []
        sim1 = simp.SimPeriod(sim_name, num_rounds, agents, 
               market, grid_size, debug, plot_on)
        for period in range(num_periods):
            sim1.run_period()
            grid = sim1.get_grid()
            sim_grids.append(grid)
            contracts.extend(sim1.get_contracts())
        
        data[week]['contracts'] = contracts
        data[week]['grids'] = sim_grids
        
        # process results
        pr1 = pr.ProcessResults(market, sim_name, agents, contracts)
        pr1.get_results()
        eff = pr1.get_efficiency()
        type_eff = pr1.get_type_surplus()
        data[week]['effs'] = eff
        data[week]['type_eff'] = type_eff
    return data


def make_monte_carlo(sim_name, num_trials, num_periods, num_weeks,
                    num_rounds, grid_size,
                    num_traders, num_units,
                    lower_bound, upper_bound,
                    trader_objects):
    """Runs one complete simulation and returns data in
        effs[treatment][trial]
    """ 

    sim_data = {}
    data['parms'] = {'sim_name': sim_name, 'num_traders': num_traders, 'num_units': num_units,
                     'num_weeks': num_weeks, 'num_periods': num_periods, 'num_rounds': num_rounds,
                     'grid_size': grid_size, 'lower_bound':lower_bound, 'upper_bound': upper_bound,
                     'trader_objects': trader_objects}

    for trial in range(num_trials):
        sim_data[trial]  = make_simulation(sim_name, num_periods, num_weeks,
                                           num_rounds, grid_size,
                                           num_traders, num_units,
                                           lower_bound, upper_bound,
                                           trader_objects)
    return sim_data



# Introduction

The user guide shows you how to use the dm_market library to run decentralized market simulations. 


## Program Flow

![Program Flow](program_flow.jpg)